Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:

In [1]:
from pydataset import data
import pandas as pd
import numpy as np


mpg = data('mpg') # load the dataset and store it in a variable
# data('mpg', show_doc=True) # view the documentation for the dataset

1a) On average, which manufacturer has the best miles per gallon?

In [9]:
overall_mpg = mpg.assign(avg_mpg_overall = (mpg.cty + mpg.hwy) / 2)
group_mpg = overall_mpg.groupby('manufacturer').agg({'avg_mpg_overall':'mean','cty':'mean','hwy':'mean'})
group_mpg.sort_values(by = ['avg_mpg_overall'], ascending = False).head(1)

,avg_mpg_overall,cty,hwy
manufacturer,,,
honda,28.5,24.444444,32.555556


1b) How many different manufacturers are there?

In [10]:
mpg.manufacturer.agg('nunique')

15

1c) How many different models are there?

In [12]:
mpg.model.agg('nunique')

38

1d) Do automatic or manual cars have better miles per gallon?

In [13]:
automan = mpg.assign(avg_mpg_overall = (mpg.cty + mpg.hwy) / 2)
automan['trans_cat'] = np.where(mpg.trans.str.startswith('a'), 'auto', 'manual')
automan_comparison = automan.groupby('trans_cat').agg({'avg_mpg_overall':'mean','cty':'mean','hwy':'mean'})

In [14]:
automan_comparison

,avg_mpg_overall,cty,hwy
trans_cat,,,
auto,19.130573,15.968153,22.292994
manual,22.227273,18.675325,25.779221


2) Copy the users and roles dataframes from the examples above. What do you think a right join would look like?

In [15]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

# copying roles DF
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
pd.merge(users, roles, left_on = 'role_id', right_on='id', how = 'right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


2) An outer join? 

In [16]:
pd.merge(users, roles, left_on = 'role_id', right_on='id', how = 'outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


2) What happens if you drop the foreign keys from the dataframes and try to merge them?

In [17]:
users = users.drop(columns = ['role_id'])
pd.merge(users, roles, left_on = 'role_id', right_on='id')

KeyError: 'role_id'

3a) Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.

3b) Use your function to obtain a connection to the employees database.

3c) Once you have successfully run a query:
Intentionally make a typo in the database url. What kind of error message do you see?
Intentionally make an error in your SQL query. What does the error message look like?

3d) Read the employees and titles tables into two separate dataframes

3e) Visualize the number of employees with each title.

3f) Join the employees and titles dataframes together.

3g) Visualize how frequently employees change titles.

3h) For each title, find the hire date of the employee that was hired most recently with that title.

3i) Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL and python/pandas code)